<a href="https://colab.research.google.com/github/j-hartmann/automated-image-analysis/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

# Installs (restart runtime after this) 

In [ ]:
%%capture
# install yolo
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r requirements.txt  # install
!cd content
!pip install deepface
!pip install retina-face
!pip install mtcnn
!pip install fpdf
# install IQA library
!pip install image-quality

In [ ]:
!pip install -U scikit-image

# Imports

In [ ]:
import cv2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
import re
import torch
from skimage import measure, segmentation
import skimage
import copy
from sklearn.preprocessing import MinMaxScaler
import PIL.Image 
import math
import time
from google.colab.patches import cv2_imshow
from deepface import DeepFace
from retinaface import RetinaFace
from mtcnn import MTCNN
from fpdf import FPDF
import imquality.brisque as brisque
import datetime

In [ ]:
# load model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Link drive (optional)

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
# create faces dir
!mkdir cropped_faces
!mkdir faces
!mkdir minmax

# Paths 
Path to either drive folder or local directory (GC) -> Need to configure yourself

In [ ]:
# path to image folder, e.g. "/content/drive/My Drive/images/" 
path_images = ""
# optional: export min/max images for each category
path_images_out_min_max = "/content/minmax/"
# path to image folder, e.g. "/content/drive/My Drive/csvs/"
path_csv_out = ""
# number of images in given path
print(len(os.listdir(path_images)))

# Control variables
Note: brisque, visualbalance (and to a lesser degree faces and demographics) will increase computation time drastically

In [ ]:
# control variables for feature extraction
start = 0 #@param {type:"integer", min:0}
end =  5#@param {type:"integer", min:0}
resize = 25 #@param {type:"integer"}
bool_brisque = False #@param ["False", "True"] {type:"raw"}
bool_visualbalance = False #@param ["False", "True"] {type:"raw"}
yolo_extraction_bool = False #@param ["False", "True"] {type:"raw"}
print_values = False #@param ["False", "True"] {type:"raw"}
rescale_w =  300#@param {type:"integer", min:0}
rescale_h = 300 #@param {type:"integer", min:0}
bool_save_min_max_img = False #@param ["False", "True"] {type:"raw"}
bool_extract_faces = True #@param ["False", "True"] {type:"raw"}
bool_demographics = True #@param ["False", "True"] {type: "raw"}
bool_mtcnn = True #@param ["False", "True"] {type: "raw"}
# sharpness is an alternative to the blur_effect values
bool_sharpness = False #@param ["False", "True"] {type: "raw"}
bool_show_face = True #@param ["False", "True"] {type: "raw"}

# Functions

In [ ]:
# NOTE: WORK IN PROGRESS

# calculate the euclidean distance between two images -> symmetrical
def euclid_dist_symmetrical(image):

  '''
  This function:
  0. Takes in an image
  1. calls helper function adjust_image_shape(img) to 
  adjust the shape of a given image if necessary
  -> when col/row uneven
  2. slices the image horizontally by height
  3. Calculates and returns the sum of all symmetrical points between both image slices
  '''

  # extract h,w
  height, width, channels = image.shape
  # exit if one of width or height are not even
  if (height % 2 != 0) or (width% 2 != 0):
    # adjust shape of image:
    # subtract one col or row or both from given image if it is not even
    # reason: sub image slice need to be of same size
    image = adjust_image_shape(image)

  # cut image vertically
  # determine value to slice image by
  slice_by = width // 2
  # slice array
  slice_1 = image[:, :slice_by]
  slice_2 = image[:, slice_by:]

  # euclidean distance of rgb tuple, symmetric
  # reference: https://en.wikipedia.org/wiki/Euclidean_distance
  euclid_distance = 0
  for index in range(len(slice_1)):
    for index2 in range(len(slice_1[index])):
        euclid_distance += math.sqrt((slice_1[index][index2][0] - slice_2[index][-index2][0])**2 + (slice_1[index][index2][1] - slice_2[index][-index2][1])**2 + (slice_1[index][index2][2] - slice_2[index][-index2][2])**2)
       
  return euclid_distance/(width*height)

In [ ]:
# NOTE: WORK IN PROGRESS

# calculate the diagonal symmetrical distance between images
def euclid_dist_diagonal(image):

  '''
  This function:
  0. Takes in an image
  1. calls helper function adjust_image_shape(img) to 
  adjust the shape of a given image if necessary
  -> when col/row uneven
  2. slices the image horizontally by height
  3. Calculates and returns the sum of all diagonals between both image slices
  '''

  # extract h,w
  height, width, channels = image.shape
  # exit if one of width or height are not even
  if (height % 2 != 0) or (width% 2 != 0):
    # adjust shape of image:
    # subtract one col or row or both from given image if it is not even
    # reason: sub image slice need to be of same size
    image = adjust_image_shape(image)

  # cut image vertically
  # determine value to slice image by
  slice_by = height // 2
  # slice array
  slice_1 = image[:slice_by, :]
  slice_2 = image[slice_by:, :]

  # euclidean distance of rgb tuple, symmetric
  # reference: https://en.wikipedia.org/wiki/Euclidean_distance
  euclid_distance = 0
  for index in range(len(slice_1)):
    for index2 in range(len(slice_1[index])):
        euclid_distance += math.sqrt((slice_1[index][index2][0] - slice_2[-index][-index2][0])**2 + (slice_1[index][index2][1] - slice_2[-index][-index2][1])**2 + (slice_1[index][index2][2] - slice_2[-index][-index2][2])**2)
  
  return euclid_distance/(width*height)

In [ ]:
# helper function for euclid_dist_diagonal/symmetrical
def adjust_image_shape(img):
  ''' 
  This function:
  0. takes in an image and its h,w
  1. if h or w of slice is uneven, subtract row or col
  2. return adjusted image
  '''

  if img.shape[0] % 2 != 0:
    # subtract row from height
    img = img[:-1, :]
  if img.shape[1] % 2 != 0:
    # subtract col from width
    img = img[:, :-1]
  return img

In [ ]:
# define MTCNN detector 
detector = MTCNN()

# helper function to detect faces in images -> RetinaFace
def _detect_faces(path_img, bool_mtcnn):
  '''
  This function:
  0. Takes in a path to an img
  1. extracts all faces (bb, landmarks) as a dict and returns it
  '''
  
  if bool_mtcnn:

    # check if image is in RGB or grayscale format
    # cv2.imread to not throw an error, therefore have to check if image has been read in
    # note: reads in image with 3 channels by default as BGR, even if grayscale
    # read image
    img = cv2.cvtColor(cv2.imread(path_img), cv2.COLOR_BGR2RGB)

    
    # predict faces
    faces = detector.detect_faces(img)

    # reformat datastructure
    # if list not empty
    faces_dict = {}
    if bool(faces):
  
      # iterate over list of dicts
      counter = 1
      for entry in faces:
        faces_dict["face_" + str(counter)] = {"score":entry["confidence"],"facial_area":entry["box"], "landmarks": {"right_eye": entry["keypoints"]["right_eye"],"left_eye":entry["keypoints"]["left_eye"], "nose":entry["keypoints"]["nose"], "mouth_right":entry["keypoints"]["mouth_right"], "mouth_left":entry["keypoints"]["mouth_left"]}}
        counter +=1

    return faces_dict
  else:
    return RetinaFace.detect_faces(path_img)

In [ ]:
# helper funciton to analyze faces -> DeepFace
def _analyze_faces(path):
  '''
  This function:
  0. Takes in a cropped face from a previously extracted face detection function
  1. analyzes it and returns the extraction information as a dict (race, sex, age)
  '''
  return DeepFace.analyze(img_path = path, detector_backend="retinaface", actions = ['age', 'gender', 'race', 'emotion'], enforce_detection=False)

In [ ]:
# class for writing pdfs of min/max images
class PDF(FPDF):
  # header of pdf
  def header(self):
    # define header of pdf
    self.set_font("Arial", "B", 20)
    self.cell(80)
    # align = C -> center
    self.cell(w=30,h=5, txt="AIA: MinMax", border=0, ln=0, align="C")
    # line break after title
    self.ln(50)
  
  # foter of pdf
  def footer(self):
    # position 2cm from bottom
    self.set_y(-20)
    # set font
    # no style -> normal
    self.set_font(family="Arial", style="", size=10)
    # number of page
    # page_no() print current page number
    self.cell(w=0, h=12, txt="" + str(self.page_no()), border=0, ln=0, align='R')

In [ ]:
def _iter(path_images, path_csv_out, resize_percent, start, end, bool_brisque, bool_visualbalance, yolo_extraction_bool,bool_mtcnn,bool_sharpness, extract_faces, demographics, print_values, bool_show_face):

  '''
  This function:
  0. [x] Takes in input/output paths 
  1. [x] reads input csv
  2. [x] iterates over given image path
  3. [x] reads images
  4. [x] performs feature extraction
  5. [x] merges extracted features with base file
  6. [x] exports new file
  7. [x] yolov5 coco inference
  8. [x] add # image features
  '''

  df_faces = pd.DataFrame(columns=["filename",
                                   "face",
                                   "score",
                                   "facial_area",
                                   "landmarks_right_eye",
                                   "landmarks_left_eye",
                                   "landmarks_nose",
                                   "landmarks_mouth_right",
                                   "landmarks_mouth_left"])
  
  df_demographics = pd.DataFrame(columns=["filename",
                                          "face",
                                          "age",
                                          "sex",
                                          "asian",
                                          "indian",
                                          "black",
                                          "white",
                                          "middle_eastern",
                                          "latino_hispanic",
                                          "angry",
                                          "disgust",
                                          "fear",
                                          "happy",
                                          "sad",
                                          "surprise",
                                          "neutral"])

  # structure new df
  df_new = pd.DataFrame(columns=["filename",
                                 "height",
                                 "width",
                                 "r_mean",
                                 "g_mean",
                                 "b_mean",
                                 "hue_avg",
                                 "saturation_avg",
                                 "brightness_avg", 
                                 "greyscale_avg",
                                 "shannon_entropy",
                                 "blur_effect_grey",
                                 "blur_effect_rgb",
                                 "felzenszwalb_unique_segments_grey",
                                 "felzenszwalb_unique_segments_rgb",
                                 "size_kB",
                                 "visual_complexity_kB",
                                 "visual_balance_euclid_dist_symmetrical",
                                 "visual_balance_euclid_dist_diagonal",
                                 "image_clarity",
                                 "sharpness",
                                 "brisque_IQA",
                                 "warm_hues_perc",
                                 "cold_hues_perc",
                                 "xmin", 
                                 "ymin",
                                 "xmax",
                                 "ymax", 
                                 "confidence", 
                                 "label"])
  
  # read names of given image directory into list
  # slice into array that we want to analyze
  names = os.listdir(path_images)[start:end]

  # faces counter
  faces_counter = 0

  # iterate over zipped data -> [(x1,y1,z1),...,(xn,yn,zn)]
  for index, name in enumerate(names):

          print(f"\n\nImage {index}/{end-start} from {start} to {end}")

          # read image as BGR
          # reads in grayscale with 3 channels per default
          try:
            img = cv2.imread(path_images + name)
          except:
            # skip image if empty file read in
            df_faces.at[faces_counter, "filename"] =  name
            df_demographics.at[faces_counter, "filename"] =  name
            df_new.at[index, "filename"] =  name
            continue


          # check if read file is not empty
          if np.shape(img) == () or img.shape == None:
            # skip image if empty file read in
            df_faces.at[faces_counter, "filename"] =  name
            df_demographics.at[faces_counter, "filename"] =  name
            df_new.at[index, "filename"] =  name
            continue

          # if faces are supposed to be extracted
          faces = {}

          # id of image
          image_id = name.split(".")[0].split("_")[-1]
          
          if extract_faces:

            # workaround, if face detector returns empty set
            faces_empty = False
            # call helper function to extract faces

            faces = _detect_faces(path_img=path_images + name, bool_mtcnn=bool_mtcnn)

            try:
              if "face_1" in faces.keys():
                faces_empty = True
                #print("detected face", faces_empty)
            except:
              print("Demographics detector unable to detect faces")

            # check if set is empty
            if faces_empty ==  True:
              # if set is not empty, iterate over detected faces and write down the collected data
              for face in faces:
                print(f"\nCurrent face (of {index}/{end-start}): {face}")
                # write filename of detected face
                df_faces.at[faces_counter, "filename"] =  name
                df_faces.at[faces_counter, "face"] = face
                # write down score of detected face
    
                df_faces.at[faces_counter, "score"] = str(faces[face]["score"])
                # write down rect values of detected face
                facial_area = faces[face]["facial_area"]
                df_faces.at[faces_counter, "facial_area"] = str(facial_area)
                # write down landmarks right eye, left eye, nose, mouthr/l
                df_faces.at[faces_counter, "landmarks_right_eye"] = str(faces[face]["landmarks"]["right_eye"])
                df_faces.at[faces_counter, "landmarks_left_eye"] = str(faces[face]["landmarks"]["left_eye"])
                df_faces.at[faces_counter, "landmarks_nose"] = str(faces[face]["landmarks"]["nose"])
                df_faces.at[faces_counter, "landmarks_mouth_right"] = str(faces[face]["landmarks"]["mouth_right"])
                df_faces.at[faces_counter, "landmarks_mouth_left"] = str(faces[face]["landmarks"]["mouth_left"])


                ## refactor: crop, write, resize and show face outside of demographics block
                # crop faces from img and save them (function requires path, not byte obj)
                tmp = img.copy()
                  # crop face out of the copy -> [y:y+h, x:x+w] 
                  # width: x2-x1, height: y2-y1
                if bool_mtcnn == False:
                    # retinaFace: x1,y1,x2,y2
                  tmp_face = tmp[facial_area[1]:facial_area[1] +facial_area[3]-facial_area[1], facial_area[0]:facial_area[0] +facial_area[2]-facial_area[0]]
                else:
                    # mtcnn: x,y,w,h
                    # so: [x:x+w, y:y+h]
                  tmp_face = tmp[facial_area[1]:facial_area[1] +facial_area[3], facial_area[0]:facial_area[0] +facial_area[2]]

                  # resize it
                tmp_face = cv2.resize(tmp_face, (224,224))
                 
                  # show image if control var is true
                if bool_show_face:
                  cv2_imshow(tmp_face)
     
                # if bool_demographics == True
                if demographics:

                   # save it
                  name_split = name.split(".")
                  path_crop = "/content/cropped_faces/" + name_split[0] + "_" + face + "." + name_split[1]
                  cv2.imwrite(path_crop, tmp_face)
                  
                  try:
                      # call demographics function
                    demographics_data = _analyze_faces(path_crop)

                      # write values into df
                    df_demographics.at[faces_counter, "filename"] = name
                    df_demographics.at[faces_counter, "face"] = face
                    df_demographics.at[faces_counter, "age"] = demographics_data["age"]
                    df_demographics.at[faces_counter, "sex"] = demographics_data["gender"]
                    df_demographics.at[faces_counter, "asian"] = demographics_data["race"]["asian"]
                    df_demographics.at[faces_counter, "indian"] = demographics_data["race"]["indian"]
                    df_demographics.at[faces_counter, "black"] = demographics_data["race"]["black"]
                    df_demographics.at[faces_counter, "white"] = demographics_data["race"]["white"]
                    df_demographics.at[faces_counter, "middle_eastern"] = demographics_data["race"]["middle eastern"]
                    df_demographics.at[faces_counter, "latino_hispanic"] = demographics_data["race"]["latino hispanic"]
                    df_demographics.at[faces_counter, "angry"] = demographics_data["emotion"]["angry"]
                    df_demographics.at[faces_counter, "disgust"] = demographics_data["emotion"]["disgust"]
                    df_demographics.at[faces_counter, "fear"] = demographics_data["emotion"]["fear"]
                    df_demographics.at[faces_counter, "happy"] = demographics_data["emotion"]["happy"]
                    df_demographics.at[faces_counter, "sad"] = demographics_data["emotion"]["sad"]
                    df_demographics.at[faces_counter, "surprise"] = demographics_data["emotion"]["surprise"]
                    df_demographics.at[faces_counter, "neutral"] = demographics_data["emotion"]["neutral"]
                  except:
                     #write values into df
                    df_demographics.at[faces_counter, "filename"] =  name
                    df_demographics.at[faces_counter, "face"] = face
                    df_demographics.at[faces_counter, "age"] = "na"
                    df_demographics.at[faces_counter, "gender"] = "na"
                    df_demographics.at[faces_counter, "asian"] = "na"
                    df_demographics.at[faces_counter, "indian"] = "na"
                    df_demographics.at[faces_counter, "black"] = "na"
                    df_demographics.at[faces_counter, "white"] = "na"
                    df_demographics.at[faces_counter, "middle_eastern"] = "na"
                    df_demographics.at[faces_counter, "latino hispanic"] = "na"
                    df_demographics.at[faces_counter, "angry"] = "na"
                    df_demographics.at[faces_counter, "disgust"] = "na"
                    df_demographics.at[faces_counter, "fear"] = "na"
                    df_demographics.at[faces_counter, "happy"] = "na"
                    df_demographics.at[faces_counter, "sad"] = "na"
                    df_demographics.at[faces_counter, "surprise"] = "na"
                    df_demographics.at[faces_counter, "neutral"] = "na"
                faces_counter += 1

          # since read in image differs in output val when read with PIL/cv2, choose PIL as stated in documentation
          pil_image = PIL.Image.open(path_images + name)

          # resize image
          output = cv2.resize(img, (int(img.shape[1] * resize_percent / 100), int(img.shape[0] * resize_percent / 100)))

          # brisque val
          brisque_val = 0
          
          # perform brisque computation
          if bool_brisque:
            try:
              brisque_val = brisque.score(output)
            except:
              print("brisque error")

            if print_values:
              print(f"brisque={brisque_val}")

          sharpness = ""
          # sharpness
          if bool_sharpness:
            sharpness = cv2.Laplacian(img, cv2.CV_64F).var()
            if print_values:
              print(f"sharpness={sharpness}")

          # get height and width
          height, width, channel = img.shape
          if print_values:
            print("height, width",height, width)
          
          # convert to RGB
          img_rgb = cv2.imread(path_images + name, cv2.COLOR_BGR2RGB)
          
          is_rgb = False
          # check if image in rgb format
          if len(img_rgb.shape) > 2:
            is_rgb = True



          # calculate the symmetrical euclidean distance of a given image
          # important: have to verify if this follows description from paper
          euclid_dist_sym_bool = True
          euclid_dist_diag_bool = True
          if bool_visualbalance:
    
            try:
              euclid_dist_sym  = euclid_dist_symmetrical(img_rgb)
            except:
              euclid_dist_sym_bool = False
              print("error: uneven image")

            # euclid dist diagonal visual balance
            # own idea, needs to be verified and testet if useful
            
            try:
              euclid_dist_diag = euclid_dist_diagonal(img_rgb)
            except:
              euclid_dist_diag_bool = False
              print("error: uneven image")

          # split image in RGB format into single color changels
          try:
            r,g,b = cv2.split(img_rgb)
            # take mean
            r_mean = r.mean()
            g_mean = g.mean()
            b_mean = b.mean()

            if print_values:
              print(f"r={r_mean}\nb={g_mean}\nb={b_mean}")

              # means of colors
            df_new.at[index, "r_mean"] = r_mean
            df_new.at[index, "g_mean"] = g_mean
            df_new.at[index, "b_mean"] = b_mean
          except:
            print("Error: unable to extract RGB-colors")
            # write na's into rgb cols if read image is not in rgb format
            df_new.at[index, "r_mean"] = "na"
            df_new.at[index, "g_mean"] = "na"
            df_new.at[index, "b_mean"] = "na"

          # convert to HSV-map
          if is_rgb:
            img_hsv = cv2.imread(path_images + name, cv2.COLOR_BGR2HSV)

            # average of values
            hue = img_hsv[0].mean()
            if print_values:
              print(f"hue mean={hue}")
            saturation = img_hsv[1].mean()
            if print_values:
              print(f"saturation mean={saturation}")
            brightness = img_hsv[2].mean()
            if print_values:
              print(f"brightness mean={brightness}")

            # calculate proportion of warm hues
            hues = img_hsv[0]
            if print_values:
              print(f"hues={hues}")
            # flatten hue values
            hues_flattened = [item for sublist in hues for item in sublist]
            # filter warm hues
            warm_hues = [item for item in hues_flattened if item >= 30 and item <= 110]
            cold_hues = [item for item in hues_flattened if item < 30 or item > 110]
            if print_values:
              print(f"warm hues={warm_hues}")
              print(f"cold hues={cold_hues}")

            # percentage of image that is of warm hue
            warm_hues_perc = len(warm_hues) / len(hues_flattened)
            if print_values:
              print(f"warm hues perc={warm_hues_perc}")
            cold_hues_perc = len(cold_hues) / len(hues_flattened)
            if print_values:
              print(f"cold_hues_perc={cold_hues_perc}")

            df_new.at[index, "hue_avg"] = hue
            df_new.at[index, "warm_hues_perc"] = warm_hues_perc
            df_new.at[index, "cold_hues_perc"] = cold_hues_perc
            df_new.at[index, "saturation_avg"] = saturation
            df_new.at[index, "brightness_avg"] = brightness
          else:
            df_new.at[index, "hue_avg"] = "na"
            df_new.at[index, "warm_hues_perc"] = "na"
            df_new.at[index, "cold_hues_perc"] = "na"
            df_new.at[index, "saturation_avg"] = "na"
            df_new.at[index, "brightness_avg"] = "na"


          
          # define scaler
          try:
            scaler = MinMaxScaler(feature_range=(0, 1))
            # normalize brightness to [0-1]
            normalize_brightness = scaler.fit_transform(img_hsv[2])
            if print_values:
              print(f"normalize_brightness={normalize_brightness}")

            # flatten nested list
            brightness_flattened = [item for sublist in normalize_brightness for item in sublist]
            if print_values:
              print(f"flattened={brightness_flattened}")

            # count bright values -> [0,1] as described in paper
            # https://pubsonline.informs.org/doi/suppl/10.1287/mnsc.2021.4175/suppl_file/mnsc.2021.4175.sm1.pdf
            bright_values = [item for item in brightness_flattened if item >= 0.7]

            # image clarity
            image_clarity = len(bright_values)/len(brightness_flattened)
            if print_values:
              print(f"image_clarity={image_clarity}")
            
            # image clarity
            df_new.at[index, "image_clarity"] = image_clarity
          except:
            print("Error: could not compute image clarity")
            # image clarity
            df_new.at[index, "image_clarity"] = "na"

          # convert to grayscale
          img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

          # calculate contrast of image
          # we choose RMS contrast, which is the SD of a gray image
          RMS_contrast = img_grey.std()
          if print_values:
            print(f"RMS_contrast={RMS_contrast}")

          # average of grayscale
          avg_grey = np.average(img_grey)
          if print_values:
            print(f"avg_gray={avg_grey}")
          
          


          # object detection inference yolov5 (choose model earlier)
          predictions = []
          if yolo_extraction_bool:
            predictions = model(img).pandas().xyxy[0]
            if print_values:
              print(f"predictions={predictions}")

            # number of detected objects -> one xmin (arbitrary value) for each found bb
            numb_objects = len(predictions["xmin"])
            if print_values:
              print(f"numb_objects={numb_objects}")        

          # file size in kB -> file read in as # in bit
          file_size_kB = int(os.path.getsize(path_images + name) /(1024 * 8))
          if print_values:
            print(f"file size in bit={file_size_kB}")

          # visual complexity kB
          visual_complexity_kB = file_size_kB / (width*height)
          if print_values:
            print(f"visual complexity kB={visual_complexity_kB}")

          # Shannon-Entropy
          try:
            shannon_entropy = measure.shannon_entropy(img_grey)
            if print_values:
              print(f"shannon-entropy={shannon_entropy}")
          except:
            print("Error: could not compute shannon entropy")

          # felzenszwalb image segmentation -> computes # of unqiue segments in given image
          # ref: https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.felzenszwalb
          felzenszwalb_unique_grey = len(np.unique(segmentation.felzenszwalb(img_grey)))
          if is_rgb:
            felzenszwalb_unique_rgb = len(np.unique(segmentation.felzenszwalb(img_rgb, channel_axis=2)))

            if print_values:
              print(f"unique segments grey = {felzenszwalb_unique_grey} and unique segments rgb = {felzenszwalb_unique_rgb}")

            df_new.at[index, "felzenszwalb_unique_segments_rgb"] = felzenszwalb_unique_rgb

          else:
            df_new.at[index, "felzenszwalb_unique_segments_rgb"] = "na"
          
      
          # calculates the blur effect of an image (0,1)
          # 0 lowest, 1 highest blur
          # ref: https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.blur_effect
          # blur on greyscale
          try:
            blur_effect_val = measure.blur_effect(img_grey)
          except:
            print("Error: could not compute blur effect")

          # blur efffect on rgb, channel_axis indicates color axis
          if is_rgb:
            blur_effect_rgb = measure.blur_effect(img_rgb, channel_axis=2)
            if print_values:
              print(f"blur effect value={blur_effect_val}")
              print(f"blur grey = {blur_effect_val} and blur rgb = {blur_effect_rgb}")

            df_new.at[index, "blur_effect_rgb"] = blur_effect_rgb
          else:
            df_new.at[index, "blur_effect_rgb"] = "na"


          # write values"
          df_new.at[index, "filename"] = name
          df_new.at[index, "greyscale_avg"] = avg_grey
          df_new.at[index, "width"] = width
          df_new.at[index, "height"] = height
          df_new.at[index, "shannon_entropy"] = shannon_entropy
          df_new.at[index, "blur_effect_grey"] = blur_effect_val
          df_new.at[index, "felzenszwalb_unique_segments_grey"] = felzenszwalb_unique_grey
          df_new.at[index, "visual_complexity_kB"] = visual_complexity_kB
          df_new.at[index, "size_kB"] = file_size_kB

          # sharpness
          df_new.at[index, "sharpness"] = sharpness

          # RMS contrast
          df_new.at[index, "RMS_contrast"] = RMS_contrast

          # brisq val
          if bool_brisque:
            df_new.at[index, "brisque_IQA"] = brisque_val
          else:
            df_new.at[index, "brisque_IQA"] = "na"

          # euclidean average distance between pixels across the 
          # symmetrical (split image) vertical line
          if euclid_dist_sym_bool and bool_visualbalance:
            df_new.at[index, "visual_balance_euclid_dist_symmetrical"] = euclid_dist_sym
          
          else:
            df_new.at[index, "visual_balance_euclid_dist_symmetrical"] = "na"
          
          # euclidean average distance of diagonals
          # if even image
          if euclid_dist_diag_bool and bool_visualbalance:
            df_new.at[index, "visual_balance_euclid_dist_diagonal"] = euclid_dist_diag
          
          else:
              df_new.at[index, "visual_balance_euclid_dist_diagonal"] = "na"
            
          # if predicitons is not empty
          if yolo_extraction_bool == True:
            #if predictions.empty() == False:
              # write most predictions in df
              df_new.at[index,"xmin"] = predictions["xmin"].to_list()
              df_new.at[index,"ymin"] = predictions["ymin"].to_list()
              df_new.at[index,"xmax"] = predictions["xmax"].to_list()
              df_new.at[index,"ymax"] = predictions["ymax"].to_list()
              df_new.at[index,"confidence"] = predictions["confidence"].to_list()
              df_new.at[index,"label"] = predictions["name"].to_list()
          else:
            df_new.at[index,"xmin"] = "na"
            df_new.at[index,"ymin"] = "na"
            df_new.at[index,"xmax"] = "na"
            df_new.at[index,"ymax"] = "na"
            df_new.at[index,"confidence"] = "na"
            df_new.at[index,"label"] = "na"

          
  # drop unnamed if existent
  if df_new.columns[0] == "Unnamed: 0":
    df_new = df_new.drop("Unnamed: 0", axis=1)

  # define date
  date = datetime.datetime.now()

  # export merged files
  df_new.to_csv(path_csv_out + f"{date.year}-{date.month}-{date.day}-Pipeline_CV_features_start_{start}_end_{end}.csv")
  df_demographics.to_csv(path_csv_out + f"{date.year}-{date.month}-{date.day}-Pipeline_CV_demographics_start_{start}_end_{end}.csv")
  df_faces.to_csv(path_csv_out + f"{date.year}-{date.month}-{date.day}-Pipeline_CV_faces_start_{start}_end_{end}.csv")

In [ ]:
# read full csv in
# with filename
def investigate(rescale_width, rescale_height, filepath, img_out):
  
  # read previously compute feature file
  df = pd.read_csv(filepath)


  # store names in list
  names = df["filename"].to_list()
  # drop non-numeric values
  if bool_visualbalance:
    df = df.drop(["Unnamed: 0","filename", "ymax", "ymin", "xmax", "xmin", "confidence", "label"], axis=1)
  else:
    df = df.drop(["Unnamed: 0","filename", "ymax", "ymin", "xmax", "xmin","visual_balance_euclid_dist_symmetrical", "visual_balance_euclid_dist_diagonal", "confidence", "label"], axis=1)
  if bool_brisque == False:
    df = df.drop("brisque_IQA", axis=1)
  if bool_sharpness == False:
    df = df.drop("sharpness", axis=1)



    # set "na" to NaN vlaues
  df.replace('na', pd.NA, inplace=True)
  # drop all rows containing NaNs -> this will eliminate grayscale images
  df.dropna(how="any", inplace=True)

  # convert values to numeric types
  df.apply(pd.to_numeric)



  if bool_save_min_max_img:
    # initialize object
    pdf = PDF()
    # create a page
    pdf.add_page()

  counter = 0
  counter_mult = 0
  # zip indices of min, max with columns in df2
  # iterate over them
  
  print(df.idxmin())
  for indexmin, indexmax, col in zip(df.idxmin(), df.idxmax(), df.columns):

    min_name = names[int(indexmin)]
    max_name = names[int(indexmax)]


    # read in min image
    imagemin = cv2.imread(path_images + min_name)
    # resize min image to default value
    image_min_scaled = cv2.resize(imagemin, (rescale_width, rescale_height))
    # read in max image
    imagemax = cv2.imread(path_images + max_name)
    # rescale max image to default value
    image_max_scaled = cv2.resize(imagemax, (rescale_width, rescale_height))

    min_val = df.at[int(indexmin), col]
    max_val = df.at[int(indexmax), col]

    double_window =  np.concatenate((image_min_scaled, image_max_scaled), axis=1)
    print(f"\nFeature:  {col}\nLeft(min) value: {min_val} image: {min_name}")
    print(f"Right(max) value: {max_val} image: {max_name}")
    cv2_imshow(double_window)


    

    # if save min/max selected
    if bool_save_min_max_img:
      # save min
      cv2.imwrite(img_out + col + "_min_" + min_name, imagemin)
      # save max
      cv2.imwrite(img_out + col + "_max_" + max_name, imagemax)


      # write values slightly above images
      pdf.set_y(20 + (counter_mult * 67.5))
      pdf.set_font('Arial', '', 10)
      pdf.cell(w=0, h = 0, txt = str(min_val), border = 0, ln = 0,align = 'L', fill = False)
      pdf.cell(w=0, h = 0, txt = str(max_val), border = 0, ln = 0,align = 'R', fill = False)


      # write images into df
      pdf.image(img_out + col + "_min_" + min_name, x=11, y=25 + (counter_mult * 67.5), w=50, h=50)
      pdf.image(img_out + col + "_max_" + max_name, x=150, y=25 + (counter_mult * 67.5), w=50, h=50)
      
      # write col on same y level
      pdf.set_y(20 + (counter_mult * 67.5))
      pdf.set_font('Arial', '', 10)
      pdf.cell(w=0, h = 0, txt = col, border = 0, ln = 0,align = 'C', fill = False, link = '')

      # increase counters
      counter += 1
      counter_mult += 1
      # if eight images have been added
      # add new page
      if counter % 4 == 0 and counter != 0:
        pdf.add_page()
        counter_mult = 0

  # write pdf
  if bool_save_min_max_img:
    pdf.output(path_csv_out + 'MinMax_images.pdf', 'F')  




# Analyze images

In [ ]:
# time function call
starttime = time.time()
# function call
_iter(path_images=path_images,
      path_csv_out=path_csv_out,
      resize_percent = resize,
      start=start,
      end=end,
      bool_brisque=bool_brisque,
      bool_visualbalance=bool_visualbalance,
      yolo_extraction_bool = yolo_extraction_bool,
      bool_mtcnn = bool_mtcnn,
      bool_sharpness=bool_sharpness,
      extract_faces = bool_extract_faces,
      demographics = bool_demographics,
      print_values=print_values,
      bool_show_face=bool_show_face)

endtime = time.time()
print(f"Elapsed time in seconds: {round(endtime-starttime, 3)}")

# Inspect extracted features (open to see outputs)

In [ ]:
# read extracted feature file
# drop cols with NaN values
# define date - change path to file if you are running this on a different day than having the previous feature extraction performed on
date = datetime.datetime.now()
df = pd.read_csv(path_csv_out + f"{date.year}-{date.month}-{date.day}-Pipeline_CV_features_start_{start}_end_{end}.csv")

# drop cols containing NaNs
if bool_brisque == False:
  df = df.drop("brisque_IQA", axis=1)
if bool_visualbalance == False:
  df = df.drop(["visual_balance_euclid_dist_symmetrical","visual_balance_euclid_dist_diagonal"], axis=1)
if bool_sharpness == False:
  df = df.drop("sharpness", axis=1)
if yolo_extraction_bool == False:
  df = df.drop(["ymax","ymin", "xmax", "xmin", "confidence", "label"], axis=1)
if df.columns[0] == "Unnamed: 0":
  df = df.drop("Unnamed: 0", axis=1)


# set "na" to NaN vlaues
df.replace('na', pd.NA, inplace=True)
# drop all rows containing NaNs -> this will eliminate grayscale images
df = df.dropna(how="any")

df


# Investigate max/min feature values (open to see outputs)
Note: Since images might be too small or too big, they will be resized to a default value of 300.

In [ ]:
# investigates last file containing extracted features
#investigate(rescale_width=rescale_w,rescale_height=rescale_h,filepath=path_csv_out + f"{date.year}-{date.month}-{date.day}-Pipeline_CV_features_start_{start}_end_{end}.csv",img_out=path_images_out_min_max)